In [18]:
import numpy as np
from astropy.io import fits


In [19]:
decamCCD = fits.open('../bricks_data/ccds-annotated-decam-dr9.fits')
mosaicCCD = fits.open('../bricks_data/ccds-annotated-mosaic-dr9.fits')
bassCCD = fits.open('../bricks_data/ccds-annotated-90prime-dr9.fits')


In [20]:
decamCCD[1].columns
dataDecam = decamCCD[1].data
dataMosaic = mosaicCCD[1].data
dataBass = bassCCD[1].data

In [26]:
expnumD = dataDecam.field('expnum')
expnumM = dataMosaic.field('expnum')
expnumB = dataBass.field('expnum')
expnum = np.concatenate((expnumB,expnumD, expnumM))
print(len(expnumB))

146792
